# Satellite-Derived Bathymetry (SDB): Data Preprocessing

This outlines explain data preprocessing workflow for Satellite-Derived Bathymetry (SDB) using Sentinel-2. The workflow transforms raw multispectral and field survey data into a clean dataset suitable for Geospatial Artificial Intelligence (GEOAI), using Machine Learning regression models.

**Workflow Steps:**
1.  **Sunglint Correction (Deglinting):** Application of the *Hedley et al. (2005) (https://doi.org/10.1080/01431160500034086)* method to remove sea-surface glint using the Near-Infrared (NIR) band. This improves the retrieval of subsurface reflectance in optically shallow waters.
2.  **Land & Cloud Masking:** Calculation of the **Modified Normalized Difference Water Index (MNDWI)** to isolate water pixels and mask out land, clouds, and other non-water artifacts.
3.  **Spatial Extraction:** Extraction of spectral reflectance values at exact coordinates corresponding to field survey sounding points (echosounder data).
4.  **Dataset Splitting:** Partitioning data into Training (70%) and Testing (30%) sets for model development.


In [2]:
import os
import glob
import numpy as np
import rasterio
import pandas as pd
import geopandas as gpd
import rasterio as rio
from tqdm import tqdm

from preprocessing.preprocessing import sunglint_correction, calculate_mndwi, extract_raster_value
from sklearn.model_selection import train_test_split


## 1. Atmospheric & Glint Correction
The following block iterates through raw Sentinel-2 imagery to perform two critical corrections:

1.  **Hedley Deglinting**: Uses the linear relationship between NIR (Band 8) and Visible bands (RGB) to subtract glint.
    * *Equation*: $R'_i = R_i - b_i (R_{NIR} - Min_{NIR})$
2.  **MNDWI Masking**: Uses Green and SWIR bands to detect water.
    * *Threshold*: Pixels with MNDWI > 0.0 are classified as water.

In [ ]:
# ==========================================
# 1. CONFIGURATION & BATCH PROCESSING
# ==========================================

# --- Configuration (Constants) ---
INPUT_DIR = r'data'
OUTPUT_DIR = r'data\corrected'
PLOT_DIR = r'data\corrected\plot'  # For regression QA/QC plots

# Processing Parameters
MNDWI_THRESHOLD = 0.0  # Threshold > 0.0 typically indicates water

# Create directories if they don't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOT_DIR, exist_ok=True)

# Find all TIF images
tif_files = glob.glob(os.path.join(INPUT_DIR, "*.tif"))
print(f"Found {len(tif_files)} images for processing.")

# Sentinel-2 Band Mapping (Metadata)
S2_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

Found 8 images for processing.


In [4]:
# --- Main Processing Loop ---
for filepath in tqdm(tif_files, desc="Processing Images"):
    filename = os.path.basename(filepath)
    output_filepath = os.path.join(OUTPUT_DIR, f"corrected_{filename}")

    try:
        with rio.open(filepath) as src:
            # 1. Read Data
            data = src.read().astype('float32')
            profile = src.profile.copy()
            
            # Validation: Ensure sufficient bands
            if src.count < 12:
                print(f"[SKIP] {filename}: Insufficient bands ({src.count}/12).")
                continue

            # 2. Define Band Indices (0-based)
            # B2(Blue)=1, B3(Green)=2, B4(Red)=3, B8(NIR)=7, B11(SWIR)=10
            idx_blue, idx_green, idx_red = 1, 2, 3
            idx_nir = 7   
            idx_swir = 10 

            # Extract Bands
            raw_bands = {
                'blue': data[idx_blue],
                'green': data[idx_green],
                'red': data[idx_red],
                'nir': data[idx_nir],
                'swir': data[idx_swir]
            }

            # 3. Apply Hedley Sunglint Correction (Visible Bands)
            # Returns clean RGB bands and saves regression plots for QC
            corrected_rgb = sunglint_correction(
                visible_bands=[raw_bands['blue'], raw_bands['green'], raw_bands['red']],
                nir_band=raw_bands['nir'],
                output_dir=PLOT_DIR,
                image_id=filename,
                plot_graphs=True
            )
            
            clean_blue, clean_green, clean_red = corrected_rgb

            # 4. Apply MNDWI Water Masking
            # Formula: (Green - SWIR) / (Green + SWIR)
            mndwi = calculate_mndwi(clean_green, raw_bands['swir'])
            water_mask = mndwi > MNDWI_THRESHOLD

            # 5. Reconstruct & Mask Final Image
            final_stack = data.copy()
            
            # Update RGB with deglinted values
            final_stack[idx_blue] = clean_blue
            final_stack[idx_green] = clean_green
            final_stack[idx_red] = clean_red
            
            # Apply Water Mask to ALL bands (Land = 0)
            for b in range(src.count):
                final_stack[b] = np.where(water_mask, final_stack[b], 0)

            # 6. Save Corrected Image
            profile.update(dtype='float32', nodata=0)
            
            with rio.open(output_filepath, 'w', **profile) as dst:
                dst.descriptions = tuple(S2_BANDS)
                dst.write(final_stack)

    except Exception as e:
        print(f"[ERROR] {filename}: {e}")

print("\nProcessing complete. Quality control plots saved in 'plot' directory.")


Processing Images:   0%|          | 0/8 [00:00<?, ?it/s]

Processing Images: 100%|██████████| 8/8 [00:10<00:00,  1.26s/it]


Processing complete. Quality control plots saved in 'plot' directory.


## 2. Training Dataset Creation
We now match the **corrected satellite imagery** with the **field survey depth data**.
* **Input**: Corrected Sentinel-2 Image (.tif) & Sounding Points (.shp)
* **Process**: Extract spectral values (features) at each sounding location.
* **Output**: Cleaned NumPy arrays (.npy) ready for Machine Learning.

In [ ]:
# ==========================================
# 2. LOAD SURVEY DATA
# ==========================================

# Configuration (Constants)
SOUNDING_PATH = r'data\sounding\sounding.shp'
# Select the specific image to match with survey data
RASTER_PATH = r'data\corrected\corrected_s2_giliketapang_2018-05-31.tif'
OUTPUT_DATASET_PATH = r'train-test dataset'
DEPTH_COL = 'z1'  # Target variable column name

# Setup
os.makedirs(OUTPUT_DATASET_PATH, exist_ok=True)

# Load Shapefile
print("Loading sounding data...")
gdf_points = gpd.read_file(SOUNDING_PATH)

print(f"Geometry Type: {gdf_points.geom_type.unique()}")
print(f"Total Sounding Points: {len(gdf_points)}")
print(gdf_points[[DEPTH_COL, 'geometry']].head())

Loading sounding data...
Geometry Type: ['Point']
Total Sounding Points: 1055
    z1                           geometry
0 -1.4  POINT Z (113.24407 -7.67952 -1.4)
1 -1.4  POINT Z (113.24416 -7.67946 -1.4)
2 -1.6  POINT Z (113.24416 -7.67946 -1.6)
3 -1.6  POINT Z (113.24422 -7.67935 -1.6)
4 -1.4  POINT Z (113.24424 -7.67926 -1.4)


In [ ]:
# ==========================================
# 3. PIXEL EXTRACTION & CLEANING
# ==========================================

print(f"Extracting features from: {os.path.basename(RASTER_PATH)}")

# 1. Extract Values
# (Assumes extract_raster_value handles reprojection internally)
X_features, y_labels = extract_raster_value(gdf_points, RASTER_PATH, DEPTH_COL)

# 2. Data Cleaning
print("Cleaning dataset (removing NaNs and Masked values)...")

# A. Remove NaNs (Invalid pixels/Outside raster extent)
mask_no_nan = ~np.isnan(X_features).any(axis=1) & ~np.isnan(y_labels)

# B. Remove Zeros (Masked Land/Clouds)
# If a pixel is 0.0 in all bands, it was masked in Step 1.
mask_water_only = np.all(X_features != 0, axis=1)

# Combined Mask
valid_mask = mask_no_nan & mask_water_only

X_clean = X_features[valid_mask]
y_clean = y_labels[valid_mask]

print(f"Original Samples: {len(X_features)}")
print(f"Valid Samples:    {len(X_clean)} (Dropped {len(X_features) - len(X_clean)})")

Extracting features from: corrected_s2_giliketapang_2018-05-31.tif
Processing: corrected_s2_giliketapang_2018-05-31.tif
  - Reprojecting points from EPSG:4326 to EPSG:32749...
  - Extracting pixel values for 1055 points...
Cleaning dataset (removing NaNs and Masked values)...
Original Samples: 1055
Valid Samples:    1054 (Dropped 1)


In [ ]:
# ==========================================
# 4. SPLIT & SAVE
# ==========================================

# 1. Train/Test Split (70/30)
print("Partitioning dataset...")
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean, test_size=0.3, random_state=42
)

# 2. Save to Disk
print("Saving artifacts...")
np.save(os.path.join(OUTPUT_DATASET_PATH, 'X_train.npy'), X_train)
np.save(os.path.join(OUTPUT_DATASET_PATH, 'X_test.npy'), X_test)
np.save(os.path.join(OUTPUT_DATASET_PATH, 'y_train.npy'), y_train)
np.save(os.path.join(OUTPUT_DATASET_PATH, 'y_test.npy'), y_test)

print("-" * 30)
print(f"Success! Dataset saved to: '{OUTPUT_DATASET_PATH}'")
print(f"Train Shape: X={X_train.shape}, y={y_train.shape}")
print(f"Test Shape:  X={X_test.shape},  y={y_test.shape}")

Partitioning dataset...
Saving artifacts...
------------------------------
Success! Dataset saved to: 'train-test dataset'
Train Shape: X=(737, 12), y=(737,)
Test Shape:  X=(317, 12),  y=(317,)
